In [1]:
import pandas as pd
import numpy as np
import matplotlib.patches as mpatches
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy import stats
import seaborn as sns; sns.set_theme(color_codes=True)
import os
import matplotlib.pyplot as plt
import sklearn.cluster
import sklearn.metrics
import sklearn.datasets
import warnings
warnings.filterwarnings("ignore")

meqtls = pd.read_csv("Carter Lab Datasets/meQTL Data/all_cancers_meQTLs.csv")
meqtls

In [16]:
meqtls

,Unnamed: 0,cancer_type,snp,snp_position,alleles,probes,probe_position,probe_gene,beta,status,...,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,chr,bp
0,0,BLCA,rs11684598,chr2:33952621,G/A,cg04131969,chr2:33951647,MYADML,-1.24,-40.26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chr2,33952621
1,1,BLCA,rs12232965,chr2:33954560,C/T,cg04131969,chr2:33951647,MYADML,-1.24,-39.59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chr2,33954560
2,2,BLCA,rs7574695,chr2:33953186,C/T,cg04131969,chr2:33951647,MYADML,-1.24,-39.35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chr2,33953186
3,3,BLCA,rs11777332,chr8:91676709,G/A,cg16814680,chr8:91681699,NaN,-1.20,-36.65,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chr8,91676709
4,4,BLCA,rs4332092,chr8:91677926,G/A,cg16814680,chr8:91681699,NaN,-1.20,-36.65,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chr8,91677926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11616294,293448,UCEC,rs1601870,chr3:14024162,T/C,cg23070574,chr3:14187308,XPC,0.42,4.58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chr3,14024162
11616295,293449,UCEC,rs10903612,chr10:2018353,C/T,cg16296438,chr10:1416569,ADARB2,0.36,4.58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chr10,2018353
11616296,293450,UCEC,rs4880953,chr10:2019373,C/T,cg16296438,chr10:1416569,ADARB2,0.36,4.58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chr10,2019373
11616297,293451,UCEC,rs11002432,chr10:79931884,C/T,cg15849445,chr10:80635973,NaN,0.58,4.58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chr10,79931884


In [7]:
common_TADs = pd.read_csv("Carter Lab Datasets/common_TADs.csv")
common_TADs = common_TADs.sort_values(["chr1", "x1"])
common_TADs = common_TADs.drop(columns=["Unnamed: 0", "Unnamed: 0.1"], axis = 1)
common_TADs

,chr1,x1,x2,chr2,y1,y2,color,f1,f2,f3,...,H3K36me3,H3K9me3,H3K4me1,H3K4me2,H3K4me3,H3K27me3,H4K20me1,KMeans_Label,Label,#meqtl
26,1,7295000,7710000,1,7295000,7710000,"255,255,0",0.81273,0.053374,0.078736,...,0.000000,0.669238,0.346034,0.267189,1.000000,0.274394,0.000000,0,0,239
10,1,7715000,8010000,1,7715000,8010000,"255,255,0",1.23070,0.045249,0.046286,...,0.225970,0.384614,0.252095,0.192944,0.274704,0.460006,0.000000,0,0,528
91,1,8030000,8410000,1,8030000,8410000,"255,255,0",1.76650,0.056312,0.052672,...,0.155612,0.642965,0.188800,0.208207,0.356119,1.000000,0.707106,1,1,245
131,1,9170000,9295000,1,9170000,9295000,"255,255,0",1.21450,0.022215,0.038104,...,1.000000,0.386305,0.183673,0.117339,0.253206,0.356945,1.000000,1,1,51
68,1,9330000,9550000,1,9330000,9550000,"255,255,0",1.51850,0.048085,0.065651,...,0.000000,0.488668,0.183505,0.200339,0.315417,1.000000,0.000000,0,0,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1088,X,108950000,109125000,X,108950000,109125000,"255,255,0",1.64140,0.051367,0.050364,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0
1097,X,109150000,109385000,X,109150000,109385000,"255,255,0",1.09450,0.040047,0.036562,...,0.000000,0.000000,0.383977,0.523487,0.000000,0.000000,1.000000,1,1,0
1095,X,117750000,117955000,X,117750000,117955000,"255,255,0",1.26780,0.032125,0.037638,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0
1091,X,133635000,133905000,X,133635000,133905000,"255,255,0",1.55090,0.059014,0.040010,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.707107,0.000000,0,0,0


In [19]:
common_TADs.iloc[3]["x2"]

9295000

In [21]:
all_snps = []
ind = 0
for i, row in common_TADs.iterrows():
    if ind % 100 == 0:
        print(ind)
    common_chr = meqtls[meqtls["chr"] == "chr"+row["chr1"]]
    overlap = common_chr[(common_chr["bp"] >= row["x1"]) & (common_chr["bp"] <= row["x2"])]
    snps = []
    snp_name = np.array(overlap["snp"])
    snp_position = np.array(overlap["snp_position"])
    for j in range(len(overlap["snp"])):
        snps.append([snp_name[j], snp_position[j]])
    all_snps.append(snps)
    ind += 1

0
100
200
300
400
500
600
700
800
900
1000
